# Chain 
- Chain은 LangChain에서 여러 컴포넌트를 연결하여 복잡한 작업을 수행할 수 있게 해주는 개념이다.
- 왜 쓰는가?
    - 간결한 코드 작성: 여러 단계의 작업을 파이프라인으로 연결하여 코드를 더 간결하게 작성할 수 있습다.
    - 재사용성: 만들어진 체인을 다른 체인의 일부로 재사용할 수 있다.
    - 모듈화: 복잡한 워크플로우를 작은 단위로 나누어 관리할 수 있다.
- 사용 방법
    - `capital_chain = prompt_template | llm | output_parser`
    - 각 컴포넌트를파이프(|) 연산자로 연결한다.
    - 그렇게 되면, 왼쪽 컴포넌트의 출력이 오른쪽 컴포넌트의 입력으로 전달된다.
    - 체인 실행은 `invoke()` 메서드를 이용하면 된다.

In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")

llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-04T14:42:27.4366521Z', 'done': True, 'done_reason': 'stop', 'total_duration': 369743700, 'load_duration': 15248300, 'prompt_eval_count': 32, 'prompt_eval_duration': 38851100, 'eval_count': 8, 'eval_duration': 315121500, 'model_name': 'llama3.2:1b'}, id='run--93ca73d0-728d-488b-b4c0-212af3d0ecbf-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template="""What is te capital of {country}?
    Return the capital name only.""",
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "France"})

ai_message = llm.invoke(prompt)

output_parser = StrOutputParser()

answer = output_parser.invoke(ai_message)


### 위의 여러 줄로 작성한 코드를 하나의 체인으로 만들어서 실행시킬 수 있다.
- `capital_chain = prompt_template | llm | output_parser`
- prompt_template 의 결과가 llm 으로 전달되고, llm 의 결과가 output_parser 로 전달된다. 실행 결과는 모두 Runnable이기 때문에 가능한 것이다.

In [31]:
capital_chain = prompt_template | llm | output_parser

In [32]:
# 체인을 실행하는 방법은 여러 가지가 있는데, 가장 쉬운 방법은 invoke 메서드를 사용하는 것이다.
# 체인의 결과는 Runnable이기 때문에 다른 Runnable을 인자로 전달할 수 있다.
capital_chain.invoke({"country": "France"})

'Paris'

### 나라 이름을 응답으로 주는 country_template를 만들어보자.

In [34]:
# country_prompt 는 나라 이름을 응답으로 주는 프롬프트이다.

country_prompt = PromptTemplate(
    template="""Guess the name of the country based on the following information:
    {information}
    Return the country name only.
    """,
    input_variables=["information"],
)

# country_chain 은 country_prompt 를 실행하고, llm 을 실행하고, output_parser 를 실행한다.
country_chain = country_prompt | llm | output_parser

# 체인을 실행하면 나라 이름을 응답으로 준다.
country_chain.invoke({"information": "This country is very famous for its wine in Europe."})


'Italy.'

### final_chain 을 만들어보자.
- final_chain 은 country_chain의 결과를 capital_chain에 전달하여 최종적으로 결과를 얻는 체인이다.

In [35]:
# country_chain의 결과를 capital_chain에 전달하여 최종 결과를 얻는다.

# 아래 코드에서 country 에는 country_chain의 결과가 저장되고, 그 값이 그대로 capital_chain에 전달된다.
final_chain = {"country": country_chain} | capital_chain

In [36]:
# final_chain 체인을 실행하면 수도 이름을 응답으로 준다.
final_chain.invoke({"information": "This country is very famous for its wine in Europe."})

'Berlin'